In [ ]:
import os
import xml.etree.ElementTree as ET
import json

def voc_to_coco(voc_dir, output_file):
    images = []
    annotations = []
    categories = []
    category_id_map = {}
    image_id = 1
    annotation_id = 1

    # Replace with your class names
    class_names = ['car', 'person', 'bicycle']

    for idx, class_name in enumerate(class_names, start=1):
        category = {'id': idx, 'name': class_name}
        categories.append(category)
        category_id_map[class_name] = idx

    for xml_file in os.listdir(voc_dir):
        if not xml_file.endswith('.xml'):
            continue

        xml_path = os.path.join(voc_dir, xml_file)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        filename = root.find('filename').text
        image_width = int(root.find('size/width').text)
        image_height = int(root.find('size/height').text)

        image = {
            'id': image_id,
            'file_name': filename,
            'width': image_width,
            'height': image_height,
        }
        images.append(image)

        for obj in root.findall('object'):
            class_name = obj.find('name').text
            bbox = obj.find('bndbox')
            x_min = int(bbox.find('xmin').text)
            y_min = int(bbox.find('ymin').text)
            x_max = int(bbox.find('xmax').text)
            y_max = int(bbox.find('ymax').text)

            category_id = category_id_map[class_name]

            annotation = {
                'id': annotation_id,
                'image_id': image_id,
                'category_id': category_id,
                'bbox': [x_min, y_min, x_max - x_min, y_max - y_min],
                'area': (x_max - x_min) * (y_max - y_min),
                'iscrowd': 0,
            }
            annotations.append(annotation)

            annotation_id += 1

        image_id += 1

    coco_data = {
        'images': images,
        'annotations': annotations,
        'categories': categories,
    }

    with open(output_file, 'w') as f:
        json.dump(coco_data, f)

# Example usage
voc_directory = '/home/gina/instseg/mask_rcnn/Mask_RCNN/ISPDataset/destination'
output_json_file = '/home/gina/instseg/mask_rcnn/Mask_RCNN/ISPDataset/json_annotations.json'
voc_to_coco(voc_directory, output_json_file)
